# Libraries and input parameters

In [1]:
# LIBRARIES #

import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

from Gafchromic import Radiochromic_RB

output_notebook()

Loading BokehJS ...

In [5]:
# INPUT PARAMETERS #

#m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/test IA/25h/'
#m_filename = 'ordreA 25h acq_no0'
m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/'
m_filename = 'etalonnage_24h_000'
m_nbOfFiles = 2
m_firstNb = 1
m_splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_normalCalib.txt'

m_imgPlotWidth = 500
m_threshold = 45000

# coefs R/B gray values:
m_coefX6 = 164192    #x6
m_coefX5 = -1141014  #x5
m_coefX4 = 3299431   #x4
m_coefX3 = -5085544  #x3
m_coefX2 = 4411686   #x2
m_coefX1 = -2046039  #x
m_coefX0 = 397570    #
m_RBmax = 1.40       # maximum R/B value over which the dose is not calculated
m_RBmin = 0.80       # minimum R/B value below which the dose is not calculated


m_dosemax = 1000.0 #cGy

# Visualization interface

In [3]:
# PLOTS IMAGE AND PROFILES #
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   imgPlotWidth: width of the image plot

def displayImage(img, sizex, sizey, imgPlotWidth):

    # Displays the dose image (p1):

    maxdose = int(np.amax(img))

    color_mapper = LinearColorMapper(palette="Viridis256", low=0, high=maxdose)

    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0),
                     title='Dose cGy')

    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")

    l1_source = ColumnDataSource(data=dict(x=[0,int(sizex)], y=[int(sizey/2),int(sizey/2)]))
    l2_source = ColumnDataSource(data=dict(x=[int(sizex/2), int(sizex/2)], y=[0, int(sizey)]))

    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    p1.line('x', 'y', source=l1_source, line_width=2, line_color=(255, 255, 255, 0.7))
    p1.line('x', 'y', source= l2_source, line_width=2, line_color=(255, 255, 255, 0.7))

    p1.add_layout(color_bar, 'right')


    # Displays the zoomed window:
    p1b_source = ColumnDataSource({'x': [], 'y': [], 'width': [], 'height': []})

    jscode="""
        var data = source.data;
        var start = cb_obj.start;
        var end = cb_obj.end;
        data['%s'] = [start + (end - start) / 2];
        data['%s'] = [end - start];
        source.change.emit();
    """

    p1.x_range.callback = CustomJS(args=dict(source=p1b_source), code=jscode % ('x', 'width'))
    p1.y_range.callback = CustomJS(args=dict(source=p1b_source), code=jscode % ('y', 'height'))

    p1b = figure(title='Zoom Window', plot_width=int(imgPlotWidth*0.5), 
                     plot_height=int(imgPlotWidth*0.5*sizey/sizex), 
                     x_range=(0,sizex), y_range=(0,sizey), tools='')
    p1b.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    rect = Rect(x='x', y='y', width='width', height='height', fill_alpha=0.1,
                    line_color='white', fill_color='white')
    p1b.add_glyph(p1b_source, rect)

    p1b.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p1b.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p1b.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p1b.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    p1b.xaxis.major_label_text_color = None  # turn off x-axis tick labels leaving space
    p1b.yaxis.major_label_text_color = None  # turn off y-axis tick labels leaving space 

    # Displays profiles:
    maxdose_x = np.amax(img[:,:])
    p2_source = ColumnDataSource(data=dict(x=np.arange(0, sizex, 1), y=img[int(sizey/2)]))
    p2 = figure(plot_width=480, plot_height=300, x_range=(0,sizex), y_range=(0,maxdose_x), 
                    title="Horizontal dose profile", toolbar_location="above")
    p2.line('x', 'y', source=p2_source, line_color="#2690d4", line_width=3, line_alpha=1.0)

    p3 = figure(plot_width=480, plot_height=300, x_range=(0,sizey), y_range=(0,maxdose_x), 
                    title="Vertical dose profile", toolbar_location="above")
    p3_source = ColumnDataSource(data=dict(x=np.arange(0, sizey, 1), y=img[:,int(sizex/2)]))
    p3.line('x', 'y', source=p3_source, line_color="#2690d4", line_width=3, line_alpha=1.0)


    # Callback functions called when the sliders are changed
    callback1 = CustomJS(args=dict(source1=l1_source, source2=p2_source), code="""
        var f = horizontalPosSlider.value;

        var data1 = source1.data;
        var y1 = data1['y'];
        y1[0] = f;
        y1[1] = f;
        source1.change.emit();

        var data2 = source2.data;
        var x2 = data2['x'];
        var y2 = data2['y'];
        for (i = 0; i < x2.length; i++) {
            y2[i] = img[f][i];
        }    
        source2.change.emit();
    """)


    callback2 = CustomJS(args=dict(source1=l2_source, source2=p3_source), code="""
        var f = verticalPosSlider.value;

        var data1 = source1.data;
        var x1 = data1['x'];
        x1[0] = f;
        x1[1] = f;
        source1.change.emit();
    
        var data2 = source2.data;
        var x2 = data2['x'];
        var y2 = data2['y'];
        for (i = 0; i < x2.length; i++) {
            y2[i] = img[i][f];
        }    
        source2.change.emit();
    """)

    # Button Callbacks:
    saveCallback = CustomJS(code="""
        console.log("hello");
    """)


    # plotting inline:
    slider1 = Slider(start=0, end=sizey, value=int(sizey/2), step=1, title="Horizontal line position", callback=callback1)
    callback1.args["horizontalPosSlider"] = slider1
    callback1.args["img"] = img

    slider2 = Slider(start=0, end=sizex, value=int(sizex/2), step=1, title="Vertical line position", callback=callback2)
    callback2.args["verticalPosSlider"] = slider2
    callback2.args["img"] = img

    crop_button = Button(label="Crop Image", button_type="default")

    save_button = Button(label="Save Image To TIFF", button_type="default", callback=saveCallback)
    saveCallback.args["img"] = img
    saveCallback.args["folder"] = m_path

    # Organizing the graphs:
    grid = gridplot([[p1,column(p1b,crop_button,save_button)],[slider1,slider2],[p2, p3]])

    show(grid)

# Main

In [6]:
# CALCULATES THE DOSE IMAGE #

try:
    g = Radiochromic_RB(m_path+m_filename, m_firstNb, m_nbOfFiles)
    rgbimg = g.getArray()
    doseimg = g.convertToDose_cubicSplineFit(m_splineFile, m_dosemax)
    size = g.getSize()
    a = g.saveToTiff(doseimg, m_path+"imgConvertedRB.tif")
    if a:
        print("file written!")
except ValueError as err:
    print('Erreur: ' + err)
    
#displayImage(rgbimg[:,:,0], size[0], size[1], m_imgPlotWidth)
#displayImage(doseimg, size[0], size[1], m_imgPlotWidth)



file written!
